# Chunking/Online Learning and Stochastic/Mini-Batch Gradient Descent


## Agenda  
- Chunking  
- Stochastic gradient descent.    
- Working with larger datasets via online learning.  
- Simple NLP. 

## Chunking
Sometimes reading the entire file into memory isn't possible. However, you might be able to summarize the data by reading it in pieces.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

dfUrl = 'https://raw.githubusercontent.com/msaricaumbc/DS_data/427906167b1370c0367c0650cc0fc465d0498d4b/ds602/Advertising.csv'

df = pd.read_csv(dfUrl, chunksize=5)
df

> Using the `chunksize` option, we created an iterator for the file instead of the typical DataFrame.

We can read over the chunks to get the data in pieces (i.e., chunks).

In [ ]:
# for chunck in df:
#     print(chunck)

In [ ]:
channels = dict()
features = ['TV', 'radio', 'newspaper', 'sales']
records = 0

i = 1
# each chunk will be 5 rows.
for chunk in df:
    for feature in features:
        channels[feature] = channels.get(feature, 0) + chunk[feature].sum()
        records += chunk.shape[0]
    print(f'Chunk {i} read. Contains {chunk.shape[0]}. Total records read: {records}.')
    i += 1
        
channels = pd.Series(channels)/records

channels[['TV', 'radio', 'newspaper']].plot.barh()
plt.title('Average of Advertising Spend by Channel', loc='left')
plt.show()

## We could also use this for mining very large text documents.

In [ ]:
from urllib import request
from zipfile import ZipFile

def download_file(file_name, url):
    res = request.urlopen(url)
    with open(file_name,'wb') as file:
        file.write(res.read())
        
def unzip(file_name, path='./'):
    # opening the zip file in READ mode 
    with ZipFile(file_name, 'r') as zip: 
        # printing all the contents of the zip file 
        zip.printdir() 

        # extracting all the files 
        print('Extracting all the files now...') 
        zip.extractall(path = path) 
        print('Done!')
        
download_file('imdb.zip', 'https://github.com/msaricaumbc/DS_data/blob/master/ds602/imdb2.zip?raw=true')
unzip('imdb.zip')

In [ ]:
!ls

In [ ]:
imdb_filename = 'imdb.csv'

In [ ]:
im = pd.read_csv(imdb_filename, chunksize=1)
next(im)

In [ ]:
next(im)

> Create a word cloud while only importing 1 row at a time.

In [ ]:
import datetime
words = dict()

im = pd.read_csv(imdb_filename, chunksize=50)

st = datetime.datetime.now()
for chunk in im:
    r = chunk['review'].str.split(' ').tolist()[0]
    for w in r:
        words[w.lower()] = words.get(w.lower(), 0) + 1

en = datetime.datetime.now()
el = en - st

print(f'Completed in {el}')

In [ ]:
print(f'Length of new dictionary: {len(words.keys()):,}')

In [ ]:
words

#### Remove the stopwords from our frequencies.

In [ ]:
import nltk
nltk.download('stopwords')

from nltk.corpus import stopwords
stop = stopwords.words('english')

In [ ]:
stop[:5]

In [ ]:
for sw in list(stop):
    words.pop(sw, None)
    
print(f'Length of new dictionary: {len(words.keys()):,}')

> Could also use this to in conjunction with mini-batches for training or scoring large datasets with limited computing resources and to inform some of the feature engineering.


## What if your data is so large that your models take hours to train?
- All of the models we've talked about so far use `batch` learning, which means the entire dataset is needed to update the parameters at each iteration.  
- Really large datasets make this unfeasible.  
- An alternative is `stochastic gradient descent (SGD)` or `mini-batch gradient descent`.  
- `SGD` is also referred to as iterative or `online learning` since it learns after 1 observation.  
- `mini-batch` learns after a batch of observations, something more than 1 and less than the number of observations in the dataset.

#### Difference from `Gradient Descent`
- Instead of updating the weights after each iteration, we update the weights based on the `cumulative sum of errors` over all the examples.  

$$
\vartriangle w = \eta \sum_{i}(y^{(i)}-\phi (z^{(i)}))x^{(i)}
$$

- $\eta$ is the learning rate.  
- `epochs` total number of times we are going to pass through the entire dataset (`outer for loop`), i.e., iterations.    
- `batches` number of observations we'll use to update the errors each time (`inner for loop`). For `SGD` this will be 1.    
- This is an approximation of gradient descent.  
- Generally will converge faster because we are updating the weights more often.  
- Since this is updated after after sample error, the error surface will be noisey.  
- While the surface is noisey, it can break out of local minima easier.  
- The trade-off to the ability to break out of local minimas is that it will can have trouble settling due to the variation that will now be in the loss curves.    
- The learning rate, $\eta$, is usually replaced by an adaptive learning rate, which usually helps reach the closer to the convergence point gradient descent would reach, i.e., the learning rate will generally decrease as epochs and iterations continue in order to better fine tune. This also helps with making sure it settles down. Usually this is implemented something like this:

$$
\frac{t_0}{epochs+t_1}
$$


- We'll want to reshuffle the data so the training examples are independently, identically distributed after each epoch (full training cycle) to prevent recurring cycles that could influence learning.  
- Another big benefit to `SGD` is that it allows us to stream observations into the model for training purposes, i.e., we'd never actually need to have all of the training data in memory for the purposes of modeling. If you data is say for example 10Gb and you only have 4Gb of memory on your machine, you wouldn't be able to load the entire dataset.

### Simple example of `SGD`
`Hands-on Machine Learning with Scikit-Learn, Keras & TensorFlow, page 125.`

In [ ]:
!pip install pyprind

#### Create sample data that we'll eventually want to run a regression on.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

X = 2 * np.random.rand(1000,1)
y = 4 + 3 * X + np.random.randn(1000,1)

plt.scatter(X, y)
plt.show()

#### Adding the constant term to $X$
Remember you might need to do this depending on the software you are using!

In [ ]:
X_b = np.c_[np.ones((1000,1)), X]
X_b[0, :]

### Simple implementation of stochastic gradient descent for regression
- Go through the dataset 25,000 times. 
- Update parameters after each observation.

In [ ]:
import numpy as np

n_epochs = 25000
m = 1
t0, t1 = 5, 50

def learning_schedule(t):
    return t0 / (t + t1)

# initialization to random numbers
# recall this will cause the results to differ each time unless we freeze the seed
theta = np.random.randn(2,1)

for epoch in range(n_epochs):
    for i in range(m):
        random_index = np.random.randint(m)
        
        # shuffling the dataset
        xi = X_b[random_index:random_index+1]
        yi = y[random_index:random_index+1]
        
        gradients = 2 * xi.T.dot(xi.dot(theta) - yi)
        eta = learning_schedule(epoch * m + i)
        theta -= eta * gradients
        
sgd_intercept, sgd_slope = theta

print(f'Intercept: {float(sgd_intercept[0]):.2f}')
print(f'Slope: {float(sgd_slope[0]):.2f}')

### Mini-batch
- Go through the dataset 500 times. 
- Update parameters after 50 observations.  

In [ ]:
import numpy as np

n_epochs = 500
m = 100
t0, t1 = 5, 50

def learning_schedule(t):
    return t0 / (t + t1)

# initialization to random numbers
# recall this will cause the results to differ each time unless we freeze the seed
theta = np.random.randn(2,1)

for epoch in range(n_epochs):
    for i in range(m):
        random_index = np.random.randint(m)
        
        xi = X_b[random_index:random_index+1]
        yi = y[random_index:random_index+1]
        
        gradients = 2 * xi.T.dot(xi.dot(theta) - yi)
        eta = learning_schedule(epoch * m + i)
        theta -= eta * gradients
        
sgd_minibatch_intercept, sgd_minibatch_slope = theta
print(f'Intercept: {float(sgd_minibatch_intercept[0]):.2f}')
print(f'Slope: {float(sgd_minibatch_slope[0]):.2f}')

### Comparison to batch model
- All observations at once

In [ ]:
from sklearn.linear_model import LinearRegression

lm = LinearRegression(fit_intercept=False).fit(X_b,y)
lm_intercept = lm.coef_[0][0]
lm_slope = lm.coef_[0][1]

print(f'Intercept: {lm_intercept:.2f}')
print(f'Slope: {lm_slope:.2f}')

> Not exact, but pretty close. The trade-off in performance relative to computation will be worthwhile for very large datasets. Mini-batch will be closer to batch results generally speaking.  

In [ ]:
import matplotlib.pyplot as plt 
import numpy as np    

def abline(intercept, slope, color, label):
    """Plot a line from slope and intercept"""
    axes = plt.gca()
    x_vals = np.array(axes.get_xlim())
    y_vals = intercept + slope * x_vals
    plt.plot(x_vals, y_vals, color, label=label)
    
    
plt.scatter(X, y)
abline(sgd_intercept, sgd_slope, 'r-', 'sgd')
abline(sgd_minibatch_intercept, sgd_minibatch_slope, 'b-', 'mini_batch')
abline(lm_intercept, lm_slope, 'g-', 'lr')
plt.legend()
plt.show()


## Stochastic Gradient Descent (SGD) in scikit-learn
- [Linear Regression $\rightarrow$ LinearRegression:SGDRegressor](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.SGDRegressor.html#sklearn.linear_model.SGDRegressor)  
- [Various Classifiers $\rightarrow$ SGDClassifier](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.SGDClassifier.html#sklearn.linear_model.SGDClassifier)  

#### SGDClassifier
The `loss` parameter controls the type of classifier you can use. The following is from the documentation:

>The possible options are `hinge`, `log`, `modified_huber`, `squared_hinge`, `perceptron`, or a regression loss: `squared_error`, `huber`, `epsilon_insensitive`, or `squared_epsilon_insensitive`.
<br><br>The `log` loss gives logistic regression, a probabilistic classifier. `modified_huber` is another smooth loss that brings tolerance to outliers as well as probability estimates. `squared_hinge` is like hinge but is quadratically penalized. `perceptron` is the linear loss used by the perceptron algorithm. The other losses are designed for regression but can be useful in classification as well; see SGDRegressor for a description.  

Use `.partial_fit` to utilize online/out-of-core learning:

In [ ]:
from sklearn.linear_model import SGDRegressor

sgd_lm = SGDRegressor(fit_intercept=False,
                      alpha=0, 
                      eta0=0.01, 
                      power_t=.05, 
                      learning_rate='adaptive'
                     )

samples = X_b.shape[0]
random_idx = np.random.choice(range(1000), size=1000, replace=False)

X_b = X_b[random_idx]
y = y[random_idx]

coefs = list()
mses = list()

for i in range(samples):
    sgd_lm = sgd_lm.partial_fit(X_b[i:i+1], y[i:i+1].ravel())

    b = sgd_lm.coef_[0]
    m = sgd_lm.coef_[1]
    coefs.append((b, m))
    
    yhat = sgd_lm.predict(X_b)
    mse = ((y - yhat)**2).mean()
    mses.append(mse)

plt.plot(range(samples), [x[0] for x in coefs])
plt.plot(range(samples), [x[1] for x in coefs])
plt.legend(['Intercept', 'Slope'])
plt.xlabel('Example')
plt.ylabel('Weight')
plt.show()

plt.plot(range(samples), mses)
plt.xlabel('Example')
plt.ylabel('Mean Squared Error')
plt.show()

> Parameter curve won't be as smooth as with batch updates.  
> See the noise in the loss curve that we haven't seen in batch gradient descent?

# Sentiment Analysis with Larger Data
- Sentiment analysis is a type of `natural language processing`.  
- We'll be trying to predict whether the text is positive or negative about the movie. The movie being reviewed is unknown.  
- We'll need to:
    - Clean the data.  
    - Extract features.  
    - Train a `batch` model.  
    - Compare to an `online` model.  
    - Look at extracting topic vectors.  
    
Data source is from [Andrew L. Maas, Raymond E. Daly, Peter T. Pham, Dan Huang, Andrew Y. Ng, and Christopher Potts. (2011). Learning Word Vectors for Sentiment Analysis. The 49th Annual Meeting of the Association for Computational Linguistics (ACL 2011).](https://ai.stanford.edu/~amaas/data/sentiment/)

In [ ]:
import pandas as pd

file_name = 'imdb.csv'

df = pd.read_csv(file_name, encoding='utf-8')
df.head()

In [ ]:
df.info()

## Bag-of-Words
We'll use a `bag-of-words` model for the prediction. Recall we've introduced this in earlier examples, such as the `spam` detection examples.

`Bag-of-words` will let us represent the text reviews as simple numerical vectors.  

- Create a `vocabulary` of unique tokens (i.e., words).  
- Construct feature vector, with the counts of how often the word appears. These will mostly be sparse vectors, since each review will only use a small percentage of the unique words across the entire dataset.

In [ ]:
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer

count = CountVectorizer()

docs = np.array(['hello there bill the ', 
                  'the ravens defense is terrible',
                  'the traffic on 695 is silly',
                  'the sun was shining today',
                  'my daughter keeps me up to all hours of the night'
                 ])

bag = count.fit_transform(docs)
print(count.vocabulary_)

> The above shows the indixes of where the words appear in the matrix

In [ ]:
bag

> Stores a sparse matrix since most of the documents only contain a small percentage of the overall vocabulary.

However, you can covert to a normal array, if needed:

In [ ]:
# print(bag.toarray()) 

#### Raw Term Frequencies
Values are also called raw term frequencies, $tf(t,d)$ - number of times a term, $t$, appears in a document, $d$.

#### Note
- Order of the words `is not` accounted for. The same vector can be created from `hello there bill` and `there bill hello`.

#### Document sparseness

In [ ]:
plt.bar(range(5), bag.toarray().mean(axis=1))
plt.ylabel('Percent of Features $=0$')
plt.xlabel('Sample of Documents, each bar is a document.')
plt.show()

#### $N-$grams
A way to add weight to the sequencing of tokens, though it explodes the size of the feature matrix if you use multiple $N-$grams.

`the sun is shining`: 1-gram $\rightarrow$ 'the', 'sun', 'is', 'shining'  
`the sun is shining`: 2-gram $\rightarrow$ 'the sun', 'sun is', 'is shining'

### Issues with text data
- Not all tokens are words, e.g., punctuation, random numbers, ...  
- Misspelled words - really need a spell-checking API, or hopefully these are rare. 
- Synonyms, generic and business-specific.  
    - Create a mapping dictionary, e.g., {'title 2':['disability', 'title ii', 'disabled worker'}
   
- Not all words are meaningful, e.g., 'the', sometimes referred to as `stop words`. These can be removed using common lists.   
- Words may not be common across a language, but are across the documents, e.g., some type of business specific word or maybe the name of a product in product reviews. We can use `term frequency-inverse document frequency`. This reduces the weights of words that occur frequency across documents.  

$$
tf-idf(t,d)=tf(t,d) \times idf(t,d), \space where \space idf(t,d)=log\frac{1 + n_d}{1+df(d,t)}
$$

In the above $n_d$ is the total number of documents, $df(d,t)$ is the documents that contain $t$.

In [ ]:
import numpy as np
from sklearn.feature_extraction.text import TfidfTransformer

tfidf = TfidfTransformer(smooth_idf=False)

tfidf = tfidf.fit_transform(bag)

print(tfidf.toarray().round(2))

#### Cleaning Text Data

In [ ]:
print('Last 50 characters of first review:\n')
df.loc[0, 'review'][-50:]

> Contains HTML tags and punctuation. Some punctuation might be meaningful, e.g., emojis.

We can create a cleaning function, that could be applied to training and test data. We aren't using any data specific to the test set, so this doesn't cause leakage. We'll do this with regular expressions, which can best be described as:

<img src='./diagrams/cat.jpeg' style='width: 500px'>

#### Regular Expressions
- Very powerful, but takes time to build complex ones.  
- See [Python's re package for lite version of documentation](https://docs.python.org/3/library/re.html).  
- For our purpose, we just want to remove the tags and standardize the text, however, regular expressions can be used to find positions in the text and extract complex features.

In [ ]:
import re

def preprocessor(text):
    # find tags and replace with empty string
    text = re.sub('<[^>]*>', '', text)
    
    # find emoticons
    emoticons = re.findall(r'(?::|;|=)(?:-)?(?:\)|\(|D|P)', text)
    
    # return the cleaned text
    text = (re.sub(r'[\W]+', ' ', text.lower()) + ' '.join(emoticons).replace('-',''))
    return text

examples = ['Hello <Br> I liked your hat :)', 
            'What a relief! That is great news',
            'Bill, thanks - I appreciate it'
           ]

for ex in examples:
    print(preprocessor(ex))

### What about `hat` versus `hats`?
In many cases the plural forms of words don't provide additional semantic meaning. It is accepted practice to form these variations to reduce the dimensionality of the data. There are several methods, including [`stemming`](https://en.wikipedia.org/wiki/Stemming) and [`lemmatization`](https://en.wikipedia.org/wiki/Lemmatisation). We'll use `stemming` since it's simplier and less computationally expensive.

In [ ]:
from nltk.stem.porter import PorterStemmer

porter = PorterStemmer()

def tokenizer_porter(text):
    return [porter.stem(word) for word in text.split()]

tokenizer_porter('the hats were great. this hat was amazing.')

### Custom Dictionaries
In business contexts, there might be multiple ways to convey the same concept, e.g., in disability programs, `disability` might be written as `t2 dib`, `title ii disabled worker`, or `oasdi`.

In [ ]:
businessTerms = {'t2 dib':'disability',
                 'title ii disabled worker':'disability', 
                 'oasdi':'disability'
                }

ex = ['I have to file for t2 dib.',
      'I have to file for title ii disabled worker.',
      'I have to file for oasdi.'
     ]

print(*ex, sep='\n')

In [ ]:
import re

for e in ex:
    for k in businessTerms.keys():
        r = re.search(k, e)
        if r:
            print(e)
            print(r)
            print(re.sub(k, businessTerms[k], e))
            print('\n')

> Not aware these types of replacements that would be useful for the IMDB data, but this could be useful in an actual business application.

# Stop Words
Stop words are generally semantically meaningless and can usually be removed for analyses that are using frequency counts.

If you haven't used `nltk` before, you might need to run:

```python
import nltk

nltk.download('stopwords')
```

In [ ]:
from nltk.corpus import stopwords

stop = stopwords.words('english')

print('Sample stop words:\n')
print(*stop[:5], sep='\n')

In [ ]:
from nltk.stem.porter import PorterStemmer

porter = PorterStemmer()

def tokenizer_porter(text, stopwords):
    return [porter.stem(word) for word in text.split() if word not in stopwords]

tokenizer_porter('the hats were great. this hat was amazing.', stop)

> See [translate from string to remove punctuation](https://stackoverflow.com/questions/265960/best-way-to-strip-punctuation-from-a-string).

# `Batch` Classifier for `imdb`
Create a training and test set. The data was already shuffled.

In [ ]:
X_train = df.loc[:24999, 'review'].values
y_train = df.loc[:24999, 'sentiment'].values

X_test = df.loc[25000:, 'review'].values
y_test = df.loc[25000:, 'sentiment'].values

print(f'Training samples: {X_train.shape[0]:,}')
print(f'Test samples: {X_test.shape[0]:,}')

Ideally, we'd want to use `grid search`, but that could take an hour to run depending on the amount of parameters we are searching over. We'll take a couple liberties and see how long it takes to train one model.

In [ ]:
import datetime
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import TfidfVectorizer

st = datetime.datetime.now()

tfidf = TfidfVectorizer(strip_accents='ascii', 
                        lowercase=True,
                        stop_words='english',
                        analyzer='word'
                       )

p = Pipeline([('vect', tfidf),
              ('lm', LogisticRegression(C=10, solver='liblinear'))
             ])

p = p.fit(X_train, y_train)

en = datetime.datetime.now()

el = en - st
print(f'Time to fit model: {el}')

In [ ]:
p_score =  p.score(X_test, y_test)
print(f'Test accuracy: {p_score:.2%}')

> Took about 4-5 seconds to run, but what if it were 25 million records instead of 25,000? Depending on your computer, 25 million records may not even fit into your RAM...

#### Online learning to the rescue!
- We can stream the documents into the model with `SGD` or `mini-batch`.  
- We will need to add some processing utilities to clean some of the streaming data. But since we are working with one or a couple dozen records at a time, that isn't a lot of processing overhead.

The tokenizer should look familiar:

In [ ]:
import numpy as np
import re
from nltk.corpus import stopwords

stop = stopwords.words('english')

def tokenizer(text):
    text = re.sub(r'<[^>]*>', '', text)
    emoticons = re.findall(r'(?::|;|=)(?:-)?(?:\)|\(|D|P)', text)
    
    text = (re.sub(r'[\W]+', ' ', text.lower()) + ' '.join(emoticons).replace('-',''))
    tokens = [w for w in text.split() if w not in stop]
    return tokens

tokenizer_test = 'hi! there bill, it is great!!!! to see you :)'
print(tokenizer(tokenizer_test))

Then we'll need a function to stream the documents (reviews).

In [ ]:
def stream_docs(path):
    with open(path, 'r', encoding='utf-8') as csv:
        next(csv)
        for line in csv:
            text, label = line[:-3], int(line[-2])
            yield text, label
            
stream = stream_docs(file_name)
next(stream)

In [ ]:
next(stream)

We'll need another function to get `mini batches` of data.

In [ ]:
def get_minibatch(doc_stream, size):
    docs, y = [], []
    try:
        for _ in range(size):
            text, label = next(doc_stream)
            docs.append(text)
            y.append(label)
    except StopIteration:
        return None, None
    return docs, y

get_minibatch(stream_docs(file_name), 20)

`CountVectorizer` requires holding the data in-memory, but `HashingVectorizer` can be used.

From [scikit-learn](https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.HashingVectorizer.html#sklearn.feature_extraction.text.HashingVectorizer):

This text vectorizer implementation uses the hashing trick to find the token string name to feature integer index mapping.

This strategy has several advantages:

- it is very low memory scalable to large datasets as there is no need to store a vocabulary dictionary in memory.  
- it is fast to pickle and un-pickle as it holds no state besides the constructor parameters.  
- it can be used in a streaming (partial fit) or parallel pipeline as there is no state computed during fit.  

There are also a couple of cons (vs using a CountVectorizer with an in-memory vocabulary):

- there is no way to compute the inverse transform (from feature indices to string feature names) which can be a problem when trying to introspect which features are most important to a model.  
- there can be collisions: distinct tokens can be mapped to the same feature index. However in practice this is rarely an issue if n_features is large enough (e.g. 2 ** 18 for text classification problems).  
- no IDF weighting as this would render the transformer stateful.

`tl:dr`: it lets you use term frequencies without holding the entire data in-memory, but you cannot use `tf-idf` or look at the vocabulary.

In [ ]:
from sklearn.feature_extraction.text import HashingVectorizer
from sklearn.linear_model import SGDClassifier

# tokenizer is the function we created a few cells above
vect = HashingVectorizer(decode_error='ignore',
                         n_features=2**21,
                         preprocessor=None,
                         tokenizer=tokenizer
                        )

clf = SGDClassifier(loss='log_loss', random_state=1)
doc_stream = stream_docs(path=file_name)

print('Ready to model!')

In [ ]:
import pyprind

iter_no = 50

pbar = pyprind.ProgBar(iter_no)

classes = np.array([0, 1])
for _ in range(iter_no):
    X_train, y_train = get_minibatch(doc_stream, size=500)
    if not X_train:
        break
    X_train = vect.transform(X_train)
    clf.partial_fit(X_train, y_train, classes=classes)
    pbar.update()

In [ ]:
X_test, y_test = get_minibatch(doc_stream, size=25000)
X_test = vect.transform(X_test)

t_scr = clf.score(X_test, y_test)
print(f'Test accuracy: {t_scr:.2%}')

> Very similar performance using `online modeling`. If the training data was actually really huge, `online` learning may have had to be our default. For our `online` model, only `500` documents were loaded into memory at any one time.

In [ ]:
import datetime

from sklearn.feature_extraction.text import CountVectorizer

st = datetime.datetime.now()

# exclude words that occur in more than 10% of the documents
# include most common 5000 words
count = CountVectorizer(stop_words='english', max_df=.1, max_features=5000)

X = count.fit_transform(df['review'].values)

from sklearn.decomposition import LatentDirichletAllocation

lda = LatentDirichletAllocation(n_components=10, 
                               random_state=10, 
                               learning_method='batch'
                              )

X_topics = lda.fit_transform(X)

en = datetime.datetime.now()

el = en-st

print(f'Time to complete: {el}')

Word importances for each of the 5,000 words for each topic:

In [ ]:
lda.components_.shape

In [ ]:
top_words = 5

feature_names = count.get_feature_names_out()

for topic_idx, topic in enumerate(lda.components_):
    print(f'Topic {topic_idx+1}')
    print(','.join([feature_names[i] for i in topic.argsort()[:-top_words - 1:-1]]))

> You would need to convert these into logical categories to present to your boss. You would want to check the topics match the text, since LDA can extract topics that aren't much stronger than random noise.

In [ ]:
comedy = X_topics[:, 0].argsort()[::-1]
for iter_idx, movie_idx in enumerate(comedy[:3]):
    print(f'\nComedy {iter_idx+1}:')
    print(df['review'][movie_idx][:300], '...')

In [ ]:
feature_names = count.get_feature_names_out()

for topic_idx, topic in enumerate(lda.components_):
    print(f'Topic {topic_idx+1}')
    print(','.join([feature_names[i] for i in topic.argsort()[:-top_words - 1:-1]]))
    
    topic = X_topics[:, topic_idx].argsort()[::-1]
    
    for iter_idx, movie_idx in enumerate(topic[:3]):
        print(f'- {iter_idx+1}:')
        print(df['review'][movie_idx][:300], '...')
        
    print ('------------')

> Seem reasonable?

# Readings and Resources
- [scikit-learn out of core learning](https://scikit-learn.org/0.15/modules/scaling_strategies.html)
- [Google's tutorial on regular expressions](https://developers.google.com/edu/python/regular-expressions)  
- [Python's regular expression documentation](https://docs.python.org/3/library/re.html)  
- [Standord NLP](https://nlp.stanford.edu/IR-book/html/htmledition/stemming-and-lemmatization-1.html)  
- [NLTK Book - Natural Language Processing with Python](https://www.nltk.org/book/)  
- [Latent Dirichlet Allocation paper by Blei and el.](https://www.jmlr.org/papers/volume3/blei03a/blei03a.pdf)  
- [IncrementialPCA](https://scikit-learn.org/stable/modules/generated/sklearn.decomposition.IncrementalPCA.html)  
- [Text Mining with R Online Textbook](https://www.tidytextmining.com)

In [ ]:
!rm imdb.zip imdb.csv